![Constantine Quantum Technologies for QFF22-Algiers](imgs/qff22_cqtech_banner.png)

# QKD challenge - Qiskit Fall Fest 2022, Algiers

### The BB84 Protocol (from Qiskit)

To make things easy on you, here's the complete code for the BB84 protocol. You may copy and modify it for your solutions bellow.

In [1]:
from qiskit import QuantumCircuit, Aer, transpile, assemble
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from numpy.random import randint
import numpy as np

In [2]:
def encode_message(bits, bases):
    message = []
    for i in range(n):
        qc = QuantumCircuit(1,1)
        if bases[i] == 0: # Prepare qubit in Z-basis
            if bits[i] == 0:
                pass 
            else:
                qc.x(0)
        else: # Prepare qubit in X-basis
            if bits[i] == 0:
                qc.h(0)
            else:
                qc.x(0)
                qc.h(0)
        qc.barrier()
        message.append(qc)
    return message

In [3]:
def measure_message(message, bases):
    backend = Aer.get_backend('aer_simulator')
    measurements = []
    for q in range(n):
        if bases[q] == 0: # measuring in Z-basis
            message[q].measure(0,0)
        if bases[q] == 1: # measuring in X-basis
            message[q].h(0)
            message[q].measure(0,0)
        aer_sim = Aer.get_backend('aer_simulator')
        qobj = assemble(message[q], shots=1, memory=True)
        result = aer_sim.run(qobj).result()
        measured_bit = int(result.get_memory()[0])
        measurements.append(measured_bit)
    return measurements

In [4]:
def remove_garbage(a_bases, b_bases, bits):
    good_bits = []
    for q in range(n):
        if a_bases[q] == b_bases[q]:
            # If both used the same basis, add
            # this to the list of 'good' bits
            good_bits.append(bits[q])
    return good_bits

In [5]:
def sample_bits(bits, selection):
    sample = []
    for i in selection:
        # use np.mod to make sure the
        # bit we sample is always in 
        # the list range
        i = np.mod(i, len(bits))
        # pop(i) removes the element of the
        # list at index 'i'
        sample.append(bits.pop(i))
    return sample

In [6]:
np.random.seed(seed=3)  # We use a known seed for RNG to make the results reproducible.
n = 100

## Step 1: Alice generates random bits.
alice_bits = randint(2, size=n)
## Step 2: Alice chooses a series of random bases: one for each bit.
alice_bases = randint(2, size=n)

## Step 3: Alice then sends a quantum message with her bits encoded in her random bases.
message = encode_message(alice_bits, alice_bases)

## Step 4: Bob chooses random bases of his own.
bob_bases = randint(2, size=n)
## Step 5: Bob then measures Alice's message in his own bases.
bob_results = measure_message(message, bob_bases)

## Step 6: Alice and Bob make their bases public, compare them, and only keep
#          the measurements where they used the same bases.
bob_key = remove_garbage(alice_bases, bob_bases, bob_results)
alice_key = remove_garbage(alice_bases, bob_bases, alice_bits)


## Step 5
sample_size = 15
bit_selection = randint(n, size=sample_size)
bob_sample = sample_bits(bob_key, bit_selection)
print("  bob_sample = " + str(bob_sample))
alice_sample = sample_bits(alice_key, bit_selection)
print("alice_sample = "+ str(alice_sample))
print("-----------")
print("  bob_key = " + str(bob_key))
print("alice_key = "+ str(alice_key))



  bob_sample = [0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1]
alice_sample = [0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1]
-----------
  bob_key = [0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1]
alice_key = [0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1]


___________________________________________________________________________________

# YOUR SOLUTIONS

### 1. PNS attack on BB84

In practice, Alice will not send single photons/qubits every single time. She will likely send multiple each time. This is because it's hard to reliably generate single photons.

In this task, Alice will always send exactly 2 photons/qubits each time.
How can Eve take advantage of this to retrieve Alice and Bob's secrete key without them knowing? Simulate this scenario using Qiskit.

In [42]:
from qiskit import QuantumCircuit, Aer, transpile, assemble
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from numpy.random import randint
import numpy as np
# Your function(s) here ...   
def encode_message(bits, bases):
    message = []
    message1=[]
    for i in range(n):
        qc = QuantumCircuit(1,1)
        qc1= QuantumCircuit(1,1)
        if bases[i] == 0: # Prepare qubit in Z-basis
            if bits[i] == 0:
                pass 
            else:
                qc.x(0)
                qc1.x(0)
        else: # Prepare qubit in X-basis
            if bits[i] == 0:
                qc.h(0)
                qc1.h(0)
            else:
                qc.x(0)
                qc1.x(0)
                qc.h(0)
                qc1.h(0)
        qc.barrier()
        qc1.barrier()
        message.append(qc)
        message1.append(qc1)
    return message,message1

In [31]:
# Your function(s) here ...   
def measure_message(message, bases):
    backend = Aer.get_backend('aer_simulator')
    measurements = []
    for q in range(n):
        if bases[q] == 0: # measuring in Z-basis
            message[q].measure(0,0)
            #message[q].measure(1,1)
        if bases[q] == 1: # measuring in X-basis
            message[q].h(0)
            #message[q].h(1)
            message[q].measure(0,0)
            #message[q].measure(1,1)
        aer_sim = Aer.get_backend('aer_simulator')
        qobj = assemble(message[q], shots=1, memory=True)
        result = aer_sim.run(qobj).result()
        measured_bit = int(result.get_memory()[0])
        measurements.append(measured_bit)
    return measurements
def remove_garbage(a_bases, b_bases, bits):
    good_bits = []
    for q in range(n):
        if a_bases[q] == b_bases[q]:
            # If both used the same basis, add
            # this to the list of 'good' bits
            good_bits.append(bits[q])
    return good_bits

def sample_bits(bits, selection):
    sample = []
    for i in selection:
        # use np.mod to make sure the
        # bit we sample is always in 
        # the list range
        i = np.mod(i, len(bits))
        # pop(i) removes the element of the
        # list at index 'i'
        sample.append(bits.pop(i))
    return sample

In [46]:
np.random.seed(seed=3)
n = 100  # Using 100 quantum states
#####   DO NOT CHANGE THE CODE ABOVE THIS LINE!   #####

## Step 1: Alice generates random bits.
alice_bits = randint(2, size=n)


## Step 2: Alice chooses a series of random bases: one for each bit.
alice_bases = randint(2, size=n)
eve_bases = randint(2, size=n)
## Step 3: Alice then sends a quantum message with her bits encoded in her random bases.
message, message_eve = encode_message(alice_bits, alice_bases)

## Step 4: Bob chooses random bases of his own.
bob_bases = randint(2, size=n)
## Step 5: Bob then measures Alice's message in his own bases.
bob_results = measure_message(message, bob_bases)
eve_results = measure_message(message_eve, eve_bases)

## Step 6: Alice and Bob make their bases public, compare them, and only keep
#          the measurements where they used the same bases.
bob_key = remove_garbage(alice_bases, bob_bases, bob_results)
alice_key = remove_garbage(alice_bases, bob_bases, alice_bits)
eve_key = remove_garbage(eve_bases, bob_bases, eve_results )


## Step 5
sample_size = 15
bit_selection = randint(n, size=sample_size)

bob_sample = sample_bits(bob_key, bit_selection)
alice_sample = sample_bits(alice_key, bit_selection)
eve_sample = sample_bits(eve_key, bit_selection)
#####   DO NOT CHANGE THE CODE BELLOW THIS LINE!   #####

# RESULTS -------------------------------------
print("  bob_sample = " + str(bob_sample))
print("alice_sample = "+ str(alice_sample))
print("  eve_sample = "+ str(eve_sample))
print("-----------")
print("  bob_key = " + str(bob_key))
print("alice_key = "+ str(alice_key))
print("  eve_key = " + str(eve_key))


  bob_sample = [1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
alice_sample = [1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
  eve_sample = [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0]
-----------
  bob_key = [0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1]
alice_key = [0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1]
  eve_key = [1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1]


#### Explain your solution (Optional)

If you did not have time to implement your code, or simply want to add further explanations, you can describe your solution in the cell bellow.<br>
(in both cases you are still expected to submit code!)

*\[REPLACE THIS WITH YOUR ANSWER\]*

*(double click here to modify)*
so technically what i thought of doing was that to declare my two qubits that are going to be the same, and measure encode them the same way (alice's way) this way eve will have the same key as alice, afterwards i was generate a base and a key for eve and that will not affect alice's and bob's since eve's message is seperate from them

______________________________

### 2. Implementing BBM92 using Qiskit

Using the functions defined above, and creating new ones, implement the BBM92 protocol.

**Reminder:** In the BBM92 protocol, a pair of entangled photons/qubits is generated. Alice receives one, and Bob receives the other.

In [ ]:
# Your function(s) here ...   


In [ ]:
# Your function(s) here ...   


In [ ]:
np.random.seed(seed=3)
n = 100  # Using 100 quantum states
#####   DO NOT CHANGE THE CODE ABOVE THIS LINE!   #####

# ...
# ...    Your solution here    ...
# ...

#####   DO NOT CHANGE THE CODE BELLOW THIS LINE!   #####

# RESULTS -------------------------------------
print("  bob_sample = " + str(bob_sample))
print("alice_sample = "+ str(alice_sample))
print("-----------")
print("  bob_key = " + str(bob_key))
print("alice_key = "+ str(alice_key))


#### Explain your solution (Optional)

If you did not have time to implement your code, or simply want to add further explanations, you can describe your solution in the cell bellow.<br>
(in both cases you are still expected to submit code!)

*\[REPLACE THIS WITH YOUR ANSWER\]*

*(double click here to modify)*

________________________________________________________

### 3. BBM92 with interception of the duplicate photons

For the same reasons as in the 1st task (with BB84), 2 pairs of photons/qubits will be generated each time instead of 1.<br>
In this scenario, Alice and Bob will receive their respective photons as in the 2nd task, and Eve will receive one of the photons from the second pair.

Show that, unlike in the BB84 case, Eve does not get any information from the additional photons after Alice and Bob make their bases public.

In [ ]:
# Your function(s) here ...   


In [ ]:
# Your function(s) here ...   


In [ ]:
np.random.seed(seed=3)
n = 100  # Using 100 quantum states
#####   DO NOT CHANGE THE CODE ABOVE THIS LINE!   #####

# ...
# ...    Your solution here    ...
# ...

#####   DO NOT CHANGE THE CODE BELLOW THIS LINE!   #####

# RESULTS -------------------------------------
print("  bob_sample = " + str(bob_sample))
print("alice_sample = "+ str(alice_sample))
print("  eve_sample = "+ str(eve_sample))
print("-----------")
print("  bob_key = " + str(bob_key))
print("alice_key = "+ str(alice_key))
print("  eve_key = " + str(eve_key))  # Shoud be different than Alice and Bob's


#### Explain your solution (Optional)

If you did not have time to implement your code, or simply want to add further explanations, you can describe your solution in the cell bellow.<br>
(in both cases you are still expected to submit code!)

*\[REPLACE THIS WITH YOUR ANSWER\]*

*(double click here to modify)*
in this challenge i thought of generating two different bits then sending each of them to alice and bob in a way that qubits are coupled which means any measurement changes that are done by the side of alice or bob will be affected to both of them, this also means that they will be the only ones having access to each others qubits and eve's one will be totally seperate 


________________________________________________________